In [1]:
from transformers import AutoTokenizer

def tok_list(tokenizer,string):
    input_ids = tokenizer(string,add_special_tokens=False)["input_ids"]
    return [tokenizer.decode(tok) for tok in input_ids]

#tokenizer = AutoTokenizer.from_pretrained("google/long-t5-local-base")
tokenizer = AutoTokenizer.from_pretrained("gpt2")
print(len(tokenizer))
print(f'Tokens von T5 für hallo: {tok_list(tokenizer,"hallo")}')

50257


2024-01-28 00:08:49.173014: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-28 00:08:49.894238: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-28 00:08:49.894297: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-28 00:08:49.894333: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-28 00:08:50.286419: I tensorflow/core/platform/cpu_feature_g

Tokens von T5 für hallo: ['hall', 'o']


In [13]:
with open("function.txt", "r", encoding="utf-8") as functions_file:
    function = functions_file.readlines()[0]
print(f'Tokens: {tok_list(tokenizer,function)}')
print(function)

Tokens: ['int', ' main', ' (', ' int', ' arg', 'c', ',', ' char', ' *', ' arg', 'v', ' [', ' ]', ' )', ' {', ' int', ' x', ' ;', ' unsigned', ' short', ' int', ' bi', 'est', ' =', ' /*', 'L', 'IT', 'ERAL', '_', 'INT', '0', 'x', '00000000', '*/', ',', ' r', ' =', ' /*', 'L', 'IT', 'ERAL', '_', 'INT', '0', 'x', '00000000', '*/', ' ;', ' long', ' s', ',', ' num', ' =', ' /*', 'L', 'IT', 'ERAL', '_', 'INT', '0', 'x', '0000', '27', '10', '*/', ' ;', ' unsigned', ' char', ' byte', '_', '2', ' =', ' /*', 'L', 'IT', 'ERAL', '_', 'CHAR', '0', 'x', '30', '*/', ' ;', ' char', ' n', 'omb', 're', '_', 'f', 'ich', ' [', ' /*', 'L', 'IT', 'ERAL', '_', 'INT', '0', 'x', '000000', '14', '*/', ' ]', ' ;', ' FILE', ' *', ' f', ' ;', ' unsigned', ' char', ' tr', 'ama', ' [', ' /*', 'L', 'IT', 'ERAL', '_', 'INT', '0', 'x', '0000', '27', '10', '*/', ' ]', ' ;', ' long', ' int', ' sig', 'u', 'ient', 'e', ',', ' total', ' =', ' /*', 'L', 'IT', 'ERAL', '_', 'INT', '0', 'x', '00000000', '*/', ' ;', ' str', 'c', 

In [2]:
from tqdm.auto import tqdm
from datasets import load_dataset
from transformers.models.gpt2.tokenization_gpt2 import bytes_to_unicode

byte_to_unicode_map = bytes_to_unicode()
unicode_to_byte_map = dict((v,k) for k, v in byte_to_unicode_map.items())
print(unicode_to_byte_map)
base_vocab = list(unicode_to_byte_map.keys()) # Nur die String-Symbole, BPE ist darauf ausgelegt, reine Strings zu verarbeiten, nicht Bytes
print(f"Größe unseres Basisvokabulars: {len(base_vocab)}")

{'!': 33, '"': 34, '#': 35, '$': 36, '%': 37, '&': 38, "'": 39, '(': 40, ')': 41, '*': 42, '+': 43, ',': 44, '-': 45, '.': 46, '/': 47, '0': 48, '1': 49, '2': 50, '3': 51, '4': 52, '5': 53, '6': 54, '7': 55, '8': 56, '9': 57, ':': 58, ';': 59, '<': 60, '=': 61, '>': 62, '?': 63, '@': 64, 'A': 65, 'B': 66, 'C': 67, 'D': 68, 'E': 69, 'F': 70, 'G': 71, 'H': 72, 'I': 73, 'J': 74, 'K': 75, 'L': 76, 'M': 77, 'N': 78, 'O': 79, 'P': 80, 'Q': 81, 'R': 82, 'S': 83, 'T': 84, 'U': 85, 'V': 86, 'W': 87, 'X': 88, 'Y': 89, 'Z': 90, '[': 91, '\\': 92, ']': 93, '^': 94, '_': 95, '`': 96, 'a': 97, 'b': 98, 'c': 99, 'd': 100, 'e': 101, 'f': 102, 'g': 103, 'h': 104, 'i': 105, 'j': 106, 'k': 107, 'l': 108, 'm': 109, 'n': 110, 'o': 111, 'p': 112, 'q': 113, 'r': 114, 's': 115, 't': 116, 'u': 117, 'v': 118, 'w': 119, 'x': 120, 'y': 121, 'z': 122, '{': 123, '|': 124, '}': 125, '~': 126, '¡': 161, '¢': 162, '£': 163, '¤': 164, '¥': 165, '¦': 166, '§': 167, '¨': 168, '©': 169, 'ª': 170, '«': 171, '¬': 172, '®': 

In [3]:

length = 60000
dataset_name = 'transformersbook/codeparrot-train'
#dataset = load_dataset(dataset_name,split="train",streaming=True)
dataset = load_dataset("codeparrot/github-code", streaming=True, split="train", languages=["C"])

iter_dataset = iter(dataset)
def batch_iterator(batch_size=10):
    for _ in tqdm(range(0,length,batch_size)):
        yield [next(iter_dataset)['code'] for _ in range(batch_size)]#yield [next(iter_dataset)['content'] for _ in range(batch_size)]
new_tokenizer = tokenizer.train_new_from_iterator(batch_iterator(), vocab_size=32768,initial_alphabet=base_vocab)


  0%|          | 0/6000 [00:00<?, ?it/s]

In [11]:
#print(tokenizer(function).tokens())


tokens = sorted(new_tokenizer.vocab.items(), key=lambda x: x[1], reverse=False)
print([f'{tokenizer.convert_tokens_to_string([t])}' for t, _ in tokens[257:280]]);

print([f'{tokenizer.convert_tokens_to_string([t])}' for t, _ in tokens[400:450]]);
     

['  ', '    ', 'in', 're', '        ', 'st', ' *', '\n\t', 'de', '**', 'er', 'at', '   ', 'on', ');', ' =', ' t', ' 0', ' (', 'en', '--', '00', '->']
[' con', 'ST', 'ul', 'li', 'il', ' -', 'ble', '32', 'ode', 'DE', ' I', 'up', ' /*', 'MA', ' g', 'OR', 'op', 'iv', 'ge', 'end', 'ype', '))', 'ate', 'ase', ' i', 'um', 'return', 'as', ' h', 'err', 'static', '\n\t\t\t', ' of', 'cl', '       ', 'ap', 'AT', 'am', 'AL', 'lock', ' F', ' is', 'SE', 'ile', ' G', 'ter', 'pr', 'AR', ' th', ' l']


In [12]:
new_tokenizer.save_pretrained("./T5Tokenizer/tokenizer")

('./T5Tokenizer/tokenizer/tokenizer_config.json',
 './T5Tokenizer/tokenizer/special_tokens_map.json',
 './T5Tokenizer/tokenizer/vocab.json',
 './T5Tokenizer/tokenizer/merges.txt',
 './T5Tokenizer/tokenizer/added_tokens.json',
 './T5Tokenizer/tokenizer/tokenizer.json')